1. Load your CSVs;

(dans le cas suivant, seloger_v10_20251211_155543.csv,seloger_v10_20251211_164416.csv)

Ces datasets ont été crées par code_scraper_v10. Ce dernier a été lancé 2 fois en vu d'augmenter la qualité du dataframe final car ces CSVs ont beaucoup de valeurs N/A, mais rarement au meme endroit.

In [1]:
import pandas as pd

df1 = pd.read_csv("Web-Scraping-Immobilier/seloger_v10_20251211_155543.csv")
df2 = pd.read_csv("Web-Scraping-Immobilier/seloger_v10_20251211_164416.csv")

2. Standardize the data

This handles different spellings of N/A, blank strings, etc.

In [2]:
# --- NORMALIZE ---
for df in (df1, df2):
    df.replace(["N/A", "NA", "", " ", None], pd.NA, inplace=True)

3. We remove exact duplicate rows.

In [3]:
# Remove exact duplicates
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

Categorise both dataframes by whether they have a URL in their URL column

In [4]:
    # Split by URL availability
df1_with = df1[df1["URL"].notna()]
df1_no   = df1[df1["URL"].isna()]
df2_with = df2[df2["URL"].notna()]
df2_no   = df2[df2["URL"].isna()]

Merge rows that HAVE URLs

In [5]:
merged_with_url = pd.merge(
    df1_with, df2_with,
    on="URL",
    how="outer",
    suffixes=("_df1", "_df2")
)


Coalesce columns (choose non-null values)

This avoids duplicate columns and fills gaps using both datasets.

We will get a clean, merged-deduped, coalesced dataset for all entries that have a URL, for both datasets where their rows have a valid URL

In [6]:
# columns to coalesce (everything except URL)
cols = [c for c in df1.columns if c != "URL"]

def coalesce(row, col):
    v1 = row.get(f"{col}_df1", pd.NA)
    v2 = row.get(f"{col}_df2", pd.NA)
    return v1 if pd.notna(v1) else v2

for col in cols:
    merged_with_url[col] = merged_with_url.apply(lambda row: coalesce(row, col), axis=1)

# Keep only final clean columns
merged_with_url_clean = merged_with_url[["URL"] + cols]


3. Fuzzy merge for rows WITHOUT URLs, where at least one of the two dataframe rows don't have a URL

(Merge if ≥3 fields match, when fields aren't N/A)

In [7]:
match_cols = [
    "Type", "Price", "Price_Per_M2", "Surface_m2",
    "Rooms", "Bedrooms", "Delivery_Date", "Address",
    "City", "PostalCode", "Department", "Program_Name"
]

def find_matches(row, df_other, threshold=3):
    comparisons = df_other[match_cols].eq(row[match_cols], axis=1)
    score = comparisons.sum(axis=1)
    return df_other[score >= threshold]

merged_no_url_rows = []

for _, row in df1_no.iterrows():
    matches = find_matches(row, df2_no, threshold=3)
    
    if len(matches) == 0:
        merged_no_url_rows.append(row.to_dict())
    else:
        for _, match in matches.iterrows():
            combined = {}
            for col in df1.columns:
                # coalesce the two rows
                v1 = row[col]
                v2 = match[col]
                combined[col] = v1 if pd.notna(v1) else v2
            merged_no_url_rows.append(combined)

merged_no_url = pd.DataFrame(merged_no_url_rows)


Combine URL and no-URL merges

In [8]:
final = pd.concat([merged_with_url_clean, merged_no_url], ignore_index=True)

# Final deduplication
final = final.drop_duplicates()


5. Export

In [15]:
# Save merged CSV with UTF-8 BOM for Excel compatibility
output_filename = "merged_clean_output.csv"
final.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"Merged CSV saved as: {output_filename} (UTF-8 BOM, Excel compatible)")


Merged CSV saved as: merged_clean_output.csv (UTF-8 BOM, Excel compatible)


6. Check final csv for duplicates/ remaining N/A values

In [16]:
import pandas as pd

# --- Load merged cleaned dataset ---
final = pd.read_csv("merged_clean_output.csv")

# --- Columns to check ---
critical_cols = ["URL","Type","Price","Price_Per_M2","Surface_m2",
                 "Rooms","Bedrooms","Address","City","PostalCode",
                 "Department","Program_Name"]

# --- Filter to only critical columns ---
df_crit = final[critical_cols].copy()

# --- Identify rows with problematic missing data ---
def is_problematic(row):
    # Check for NAs in critical columns
    crit_na = row.isna()
    
    # If Price or Price_Per_M2 has data AND City/Address/PostalCode has data → not a problem
    price_ok = pd.notna(row["Price"]) or pd.notna(row["Price_Per_M2"])
    location_ok = pd.notna(row["City"]) or pd.notna(row["Address"]) or pd.notna(row["PostalCode"])
    
    # Row is problematic if there are any NAs in critical columns AND not enough data
    if price_ok and location_ok:
        return False
    # Else, check if any critical field is missing
    return crit_na.any()

rows_problematic = df_crit[df_crit.apply(is_problematic, axis=1)]

print(f"🔹 Total rows with problematic missing data: {len(rows_problematic)}\n")
print(rows_problematic)

# --- Check for duplicates (URL or full critical columns) ---
duplicate_urls = final[final.duplicated(subset=["URL"], keep=False)]
print(f"\n🔹 Duplicate URLs (total {len(duplicate_urls)}):\n", duplicate_urls)

duplicate_rows = final[final.duplicated(subset=critical_cols, keep=False)]
print(f"\n🔹 Exact duplicate rows (critical columns, total {len(duplicate_rows)}):\n", duplicate_rows)



🔹 Total rows with problematic missing data: 614

                                                     URL         Type Price  \
1      https://www.bellesdemeures.com/205139579/detai...       Maison   NaN   
183    https://www.bellesdemeures.com/249173485/detai...  Appartement   NaN   
186    https://www.bellesdemeures.com/249202237/detai...  Appartement   NaN   
236    https://www.bellesdemeures.com/250118407/detai...  Appartement   NaN   
239    https://www.bellesdemeures.com/250131653/detai...  Appartement   NaN   
...                                                  ...          ...   ...   
11327                                                NaN          NaN   NaN   
11328                                                NaN          NaN   NaN   
11329                                                NaN          NaN   NaN   
11330                                                NaN          NaN   NaN   
11331                                                NaN          NaN   NaN   

  

6.B : Optional: save report 

In [ ]:

report_file = "merged_clean_report.xlsx"
with pd.ExcelWriter(report_file) as writer:
    rows_problematic.to_excel(writer, sheet_name="Problematic_Rows", index=False)
    duplicate_urls.to_excel(writer, sheet_name="Duplicate_URLs", index=False)
    duplicate_rows.to_excel(writer, sheet_name="Duplicate_Rows", index=False)

print(f"\n✅ Report saved as '{report_file}'. Review problematic rows and duplicates there.")